# Reacher


In [1]:
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from unityagents import UnityEnvironment

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
env = UnityEnvironment(file_name='./Reacher_Windows_x86_64/Reacher.exe')
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


### Method to plot the progress of the agent's score

In [3]:
def plot_result(scores):
    # plot the scores
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

# Done
if average among 20 agents > 30 for the last 100 episodes

In [4]:
def validate(agent, max_t, train_mode=True):
    env_info = env.reset(train_mode=train_mode)[brain_name]
    agent.reset()
    states = env_info.vector_observations
    scores_one_episode = np.zeros(states.shape[0])
    for t in range(max_t):
        actions = agent.act(states, add_noise=False)             # select an action (for each agent)
        env_info = env.step(actions)[brain_name]                 # send all actions to the environment
        next_states = env_info.vector_observations               # get next state (for each agent)
        rewards = env_info.rewards                               # get reward (for each agent)
        dones = env_info.local_done                              # see if episode finished
        scores_one_episode += env_info.rewards                   # update the score (for each agent)
        states = next_states                                     # roll over states to next time step
        if np.any(dones):                                        # exit loop if episode finished
            break
    return scores_one_episode

In [12]:
from agents.utils import ReplayBuffer, ParameterNoise, ActionNoise
from agents.parallel_ddpg import ParallelDDPG
from agents.models_success import Actor, Critic

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
torch.set_num_threads(3)
random_seed = 0
states = env_info.vector_observations
action_size = brain.vector_action_space_size
memory = ReplayBuffer(action_size, device, int(1e5), 64, random_seed)
agent = ParallelDDPG(states.shape[1], action_size, random_seed, states.shape[0], memory, Actor, Critic, device, 
                    TAU=1e-3,
                    UPDATE_EVERY=20,
                    TRANSFER_EVERY=1,
                    UPDATE_LOOP=10,
                    ADD_NOISE_EVERY=1,
                    BOOTSTRAP_SIZE=4,
                    LR_CRITIC = 1e-4,
                    LR_ACTOR = 1e-3)
agent.load("success_2/2200_final_1000_steps")
scores_one_episode = validate(agent, 900, train_mode=False)
print(scores_one_episode)

I:\MyDev\Anaconda3\envs\drlnd\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[28.81999936 35.6299992  31.3699993  34.75999922 34.19999924 32.47999927
 33.32999926 29.35999934 34.77999922 28.01999937 33.97999924 34.56999923
 33.83999924 33.60999925 33.96999924 27.35999939 32.40999928 34.14999924
 29.38999934 34.14999924]


In [ ]:
scores = []
for _ in range(100):
    scores_one_episode = validate(agent, 1000, train_mode=True)
    average = np.average(scores_one_episode)
    print("\t{}".format(average))
    scores.append(average)
plot_result(scores)
print("average in 100 episodes is: {}".format(np.average(scores)))